<a href="https://colab.research.google.com/github/LukaLujan/word2vec/blob/main/gthb3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a third part of our text classification. Here we will do combination , we will use Word2Vec pretrained model - it has 3 million of vectors(for each word) , and we will use Word2Vec network to train it on our corpus. 
The task is simple. We will do word vectorization for words that are part of our trained model, and it some word from train/test data is not part of our vocab, we check if that word is part of a Word2Vec pretrained model and take vectors from there. If a token(word) is not part of both vocabs(our trained and pretrained) we will replace with with vector of the ones. 
We have an issue here - because this whole process is taking huge amount of RAM , we will only be able to do preprocessing here. We will convert our data to numpy array. We will do both for test data and train data. We can save that data, and load it in next colab notebook where we will have "fresh RAM" for our job.


In [ ]:
#most of the process is similar like in first two notebooks. So I will comment only what is different.

import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
import gensim.downloader as api
from google.colab import drive

import gensim
from gensim.models import word2vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


'2.8.0'

In [ ]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
#Here I am importing  first model.

EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'
word2vec_model1 = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [ ]:
df =pd.read_csv("train.csv")

In [ ]:
df_text =df[['text']].copy()

In [ ]:
df=df.dropna()
df.reset_index(inplace=True)
print(len(df))

18285


In [ ]:
df_text=df_text.dropna()
df_text.reset_index(inplace=True)
print(len(df_text))

20761


In [ ]:
import re
STOPWORDS = set(stopwords.words('english'))

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer=WordNetLemmatizer()

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
#This preprocessing is for the second model, that will train it's vectors on our corpus.

messages = []

tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
for par in df_text["text"].values:
    tmp = []
    sentences = nltk.sent_tokenize(par)
    for sent in sentences:
        sent = sent.lower()
        tokens = tokenizer.tokenize(sent)
        filtered_words = [w.strip() for w in tokens if w not in STOPWORDS and len(w) > 1]
        filtered_words2 = [lemmatizer.lemmatize(w) for w in filtered_words]
        tmp.extend(filtered_words2)
         
    messages.append(tmp)

In [ ]:
#This is second model. It will be trained on our corpus.

#Dimension of vectors we are generating
EMBEDDING_DIM = 300

#Creating Word Vectors by Word2Vec Method (takes time...)
word2vec_model2 = gensim.models.Word2Vec(sentences=messages, size=EMBEDDING_DIM, window=5, min_count=3)

In [ ]:
len(word2vec_model1.vocab)

3000000

In [ ]:
#Compare the size of those.
len(word2vec_model2.wv.vocab)

70967

9032

In [ ]:
#Now we are preprocessing "titles" that will be converted into numbers and they will go into our training model.
#As we said before, computer doesn't recognize "photos or words" - it understands only numbers :) 
messages2 = []

tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
for par in df["title"].values:
    tmp = []
    sentences = nltk.sent_tokenize(par)
    for sent in sentences:
        sent = sent.lower()
        tokens = tokenizer.tokenize(sent)
        filtered_words = [w.strip() for w in tokens if w not in STOPWORDS and len(w) > 1]
        filtered_words2 = [lemmatizer.lemmatize(w) for w in filtered_words]
        tmp.extend(filtered_words2)
         
    messages2.append(tmp)

In [ ]:
def ones_vector(d):
  return np.ones(d)

In [ ]:
for i in range(len(messages2)):
 messages2[i] =" ".join(word for word in messages2[i])  

In following cell there are some changes. As you can see, I introduced "elif" in the loop. Logic is simple. If the token is not part of our trained model vocab, it will get a vector from pretrained model(that has 3 million of vectors). Else - vector of the ones will replace all unknown words. 
Also I introduced few new variables that will count how many vectors from each model is replaced. It will give us some insights what we can expect.

In [ ]:


lista_vectora =[]
trained_words=0
pretrai_words=0
ones_words   =0

for row in messages2:
  temp_ls_vec=[]
  for token in row.split():
    if token in word2vec_model2.wv.vocab:
      temp_ls_vec.append(word2vec_model2.wv[token])
      trained_words +=1
    elif token in word2vec_model1.vocab:
      temp_ls_vec.append(word2vec_model1[token])
      pretrai_words +=1
    else:
      temp_ls_vec.append(ones_vector(300))
      ones_words +=1
  lista_vectora.append(temp_ls_vec)

In [ ]:
print(f" number of trained words vectorizerd  {trained_words},  number of words from downloaded model {pretrai_words},  number of ones vectors {ones_words}")

 number of trained words vectorizerd  159001,  number of words from downloaded model 432,  number of ones vectors 1012


As you can see, most of the words were replaced by our own trained vectors(159001). Only 0.3% of the words were replaced by words from pretrained model(432) , while number of ones is 1012. How much of 0.3% new vectors will have impact on our results?

In [ ]:
max_len =[]
for sent in lista_vectora:
  max_len.append(len(sent))
print(max(max_len))
print(len(max_len))
L = max(max_len)

47
18285


In [ ]:
def null_vector(num):
  return np.zeros(num)

In [ ]:
padded_embeddings = []
for row in lista_vectora:
  temp_zer=[]
  if len(row) <L:
    for i in range(L-len(row)):
      temp_zer.append(null_vector(300))
  padded_embeddings.append(temp_zer)  

In [ ]:
padded_vectors= [ k+v for k,v in zip(lista_vectora ,padded_embeddings )]

In [ ]:
y_train_extra = np.array(df["label"])
X_train_extra = np.array(padded_vectors)

In [ ]:
#We are saving those numpy arrays into npy file, and this is an easy way to import it in next notebook.
np.save("X_train_extra.npy", X_train_extra)
np.save("y_train_extra.npy", y_train_extra)

In [ ]:
#increasing RAM on google colab!! we are deleting all variables that we don't need.

In [ ]:
del y_train_extra, X_train_extra, messages, messages2, df, padded_vectors, lista_vectora, padded_embeddings

In [ ]:
%ls

 1-s2.0-S1057521915001477-main.pdf       submit.csv         X_test.npy
'Colab Notebooks'/                       test.csv           X_train_extra.npy
 dario/                                  train.csv          y_array.npy
 GoogleNews-vectors-negative300.bin.gz   X_array.csv.npy    y_test_extra.npy
 iv.docx                                 X_array.npy        y_test.npy
 spam.csv                                X_test_extra.npy   y_train_extra.npy


In [ ]:
#we are doing preprocessing of the test data as well!
df_test =pd.read_csv("test.csv")

In [ ]:
y_t= pd.read_csv("submit.csv")

In [ ]:
result = pd.concat([df_test, y_t], axis=1)

In [ ]:
len(result)

5200

In [ ]:
result.dropna(inplace=True)

In [ ]:
len(result)

4575

In [ ]:
messagesX = []

tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
for par in result["title"].values:
    tmp = []
    sentences = nltk.sent_tokenize(par)
    for sent in sentences:
        sent = sent.lower()
        tokens = tokenizer.tokenize(sent)
        filtered_words = [w.strip() for w in tokens if w not in STOPWORDS and len(w) > 1]
        filtered_words2 = [lemmatizer.lemmatize(w) for w in filtered_words]
        tmp.extend(filtered_words2)
    messagesX.append(tmp)

In [ ]:
messagesX[0]

['specter',
 'trump',
 'loosens',
 'tongue',
 'purse',
 'string',
 'silicon',
 'valley',
 'new',
 'york',
 'time']

In [ ]:
for i in range(len(messagesX)):
 messagesX[i] =" ".join(word for word in messagesX[i])  

In [ ]:
trained_words=0
pretrai_words=0
ones_words   =0
lista_vectora2 =[]
for row in messagesX:
  temp_ls_vec=[]
  for token in row.split():
    if token in word2vec_model2.wv.vocab:
      temp_ls_vec.append(word2vec_model2.wv[token])
      trained_words+=1
    elif token in word2vec_model1.vocab:
      temp_ls_vec.append(word2vec_model1[token])
      pretrai_words +=1
    else:
      temp_ls_vec.append(ones_vector(300))
      ones_words +=1
  lista_vectora2.append(temp_ls_vec)

In [ ]:
print(trained_words, pretrai_words, ones_words)

39130 164 442


In [ ]:
padded_embeddings2 = []
for row in lista_vectora2:
  temp_zer=[]
  if len(row) <L:
    for i in range(L-len(row)):
      temp_zer.append(null_vector(300))
  padded_embeddings2.append(temp_zer)

In [ ]:
padded_vectors2= [ k+v for k,v in zip(lista_vectora2 ,padded_embeddings2 )]

In [ ]:
#converting test data into numpy arrays
y_test_extra = np.array(result["label"])
X_test_extra = np.array(padded_vectors2)

In [ ]:
len(y_test_extra) == len(X_test_extra)

True

In [ ]:
#saving test data for next notebook.
np.save("X_test_extra.npy", X_test_extra)
np.save("y_test_extra.npy", y_test_extra)

In [ ]:
X_test_extra.shape

(4575, 47, 300)